code_1

In [1]:
from ultralytics import YOLO
import cv2
import torch

# Load YOLOv8 model (pretrained on COCO dataset)
model = YOLO("yolov8n.pt")

# Load image
image_path = "test_2.jpg"  # Replace with your image path
image = cv2.imread(image_path)

# Perform object detection
results = model(image)

# Process results
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
        conf = box.conf[0].item()  # Confidence score
        cls = int(box.cls[0].item())  # Class index
        label = model.names[cls]  # Class name

        # Draw bounding box
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, f"{label} {conf:.2f}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Show output image

image_resized = cv2.resize(image, (700, 700))

# Show resized image
cv2.imshow("YOLOv8 Detection (300x300)", image_resized)
cv2.waitKey(0)
cv2.destroyAllWindows()



0: 640x480 13 persons, 2 chairs, 1 dining table, 2 laptops, 251.6ms
Speed: 5.0ms preprocess, 251.6ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 480)


code_2

In [ ]:
import cv2
import time
from ultralytics import YOLO

# Load YOLOv8 model (pretrained on COCO dataset)
model = YOLO("yolov8n.pt")

# Open webcam
cap = cv2.VideoCapture(1)  # Change to 1 or 2 if using an external webcam

# Set capture interval (every 2 seconds)
capture_interval = 2
last_capture_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()  # Read frame from webcam

    if not ret:
        print("Failed to capture frame")
        break

    # Check if 2 seconds have passed
    current_time = time.time()
    if current_time - last_capture_time >= capture_interval:
        last_capture_time = current_time  # Reset timer
        
        # Perform YOLOv8 detection
        results = model(frame)

        # Process results and draw bounding boxes
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
                conf = box.conf[0].item()  # Confidence score
                cls = int(box.cls[0].item())  # Class index
                label = model.names[cls]  # Class name

                # Draw bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Show frame
    cv2.imshow("YOLOv8 Live Detection", frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


code_3


In [ ]:
import cv2
import torch
import threading
import queue
import speech_recognition as sr
import pyttsx3
from ultralytics import YOLO
from transformers import pipeline
from PIL import Image
import numpy as np
import time

# Constants
YOLO_MODEL = 'yolov8n.pt'
RELEVANT_CLASSES = ['cell phone', 'bottle', 'laptop', 'mouse']
CALIBRATION_DISTANCE = 1.0  # meters
FRAME_SIZE = 384  # Common size for both models

# Global variables
latest_frame = None
scaling_factor = None
frame_queue = queue.Queue(maxsize=1)

# Initialize Text-to-Speech
engine = pyttsx3.init()

# Load models
yolo_model = YOLO(YOLO_MODEL)
depth_estimator = pipeline('depth-estimation', model='Intel/dpt-hybrid-midas')

def camera_capture():
    cap = cv2.VideoCapture(1)
    while True:
        ret, frame = cap.read()
        if ret:
            resized_frame = cv2.resize(frame, (FRAME_SIZE, FRAME_SIZE))
            if frame_queue.full():
                frame_queue.get_nowait()
            frame_queue.put(resized_frame)

def process_frame():
    global scaling_factor
    while True:
        frame = frame_queue.get()

        # Convert BGR (OpenCV) to RGB for transformer model
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_pil = Image.fromarray(frame_rgb)

        # Object Detection
        results = yolo_model(frame)[0]
        detections = []
        for box in results.boxes:
            class_id = int(box.cls)
            label = yolo_model.names[class_id]  # Fix applied
            if label in RELEVANT_CLASSES:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                detections.append((label, (x1, y1, x2, y2)))

        # Depth Estimation
        depth_result = depth_estimator(frame_pil)
        depth_map = depth_result['depth']  # Fix applied

        # Convert to numpy if needed
        depth_map = np.array(depth_map)

        # Process detections
        feedback = []
        for label, bbox in detections:
            x1, y1, x2, y2 = bbox
            depth_roi = depth_map[y1:y2, x1:x2]
            avg_depth = depth_roi.mean() if depth_roi.size > 0 else 0

            if scaling_factor and avg_depth > 0:
                distance = 1 / (avg_depth * scaling_factor)
                feedback.append(f"{label} {distance:.1f} meters away")
            else:
                feedback.append(f"{label} detected")

        # Audio feedback
        if feedback:
            engine.say('. '.join(feedback))
            engine.runAndWait()

import speech_recognition as sr

def voice_command_listener():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        while True:
            print("Listening...")
            audio = r.listen(source)
            try:
                text = r.recognize_google(audio).lower()
                print(f"You said: {text}")  # Print whatever is spoken
                
                if "yah kya hai" in text:
                    threading.Thread(target=process_frame, daemon=True).start()
                elif "calibrate" in text:
                    calibrate_depth()
            except Exception as e:
                print(f"Error: {e}")

# Start listening
# voice_command_listener()


def calibrate_depth():
    global scaling_factor
    if not frame_queue.empty():
        frame = frame_queue.get()
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_pil = Image.fromarray(frame_rgb)

        results = yolo_model(frame)[0]
        for box in results.boxes:
            class_id = int(box.cls)
            label = yolo_model.names[class_id]
            if label in RELEVANT_CLASSES:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                depth_result = depth_estimator(frame_pil)
                depth_map = depth_result['predictions']

                depth_map = np.array(depth_map)
                depth_roi = depth_map[y1:y2, x1:x2]
                avg_depth = depth_roi.mean() if depth_roi.size > 0 else 0

                if avg_depth > 0:
                    scaling_factor = 1 / (avg_depth * CALIBRATION_DISTANCE)
                    engine.say("Calibration complete")
                    engine.runAndWait()
                    return

# Start threads
camera_thread = threading.Thread(target=camera_capture, daemon=True)
voice_thread = threading.Thread(target=voice_command_listener, daemon=True)

camera_thread.start()
voice_thread.start()

# Keep main thread alive efficiently
while True:
    time.sleep(1)


code_4